In [8]:
import numpy as np
import random

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

In [2]:
# Load the RIR data and labels 
human_rir_data = np.load('/home/onyxia/work/hackathon-acoustic-2024/data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy') 
human_rir_labels = np.load('/home/onyxia/work/hackathon-acoustic-2024/data/LivingRoom_preprocessed_hack/Human1/centroid.npy')
print(f"Shape of centroids (labels) = {human_rir_labels.shape}")
print(f"Shape of deconvoled (features)= {human_rir_data.shape}")

Shape of centroids (labels) = (1000, 2)
Shape of deconvoled (features)= (1000, 4, 667200)


In [3]:
def rms(y): # https://stackoverflow.com/questions/40963659/root-mean-square-of-a-function-in-python
    return np.sqrt(np.mean(y**2))

rms_levels = np.apply_along_axis(rms, axis=2, arr=human_rir_data)
# Check the shape of the resulting array, just to be sure
print(rms_levels.shape)
for i in range(3):
    print(f"RMS of RIR = {rms_levels[i]}")


(1000, 4)
RMS of RIR = [0.00052505 0.00050232 0.00070836 0.00042991]
RMS of RIR = [0.00052084 0.00049965 0.00070709 0.0004291 ]
RMS of RIR = [0.00052198 0.00049891 0.00070534 0.00042922]


In [4]:
def rms_over_n_partitions(y, n):
    array_len = len(y)
    # Compute the length of each fraction
    fraction_len = array_len // n
    # Compute RMS for each fraction of the array
    rms_fractions = []
    for i in range(n):
        start_index = i * fraction_len
        end_index = (i + 1) * fraction_len if i < n - 1 else array_len
        rms_fraction = np.sqrt(np.mean(y[start_index:end_index]**2))
        rms_fractions.append(rms_fraction)
    # Concatenate the results horizontally
    return np.hstack(rms_fractions)

# Example usage:
n = 2  # Number of fractions, change this number
# Calculate RMS levels for each sample along the last axis
rms_levels = np.apply_along_axis(rms_over_n_partitions, axis=2, arr=human_rir_data, n=n)
# Check the shape of the resulting array
print(rms_levels.shape)
# Print some example values
for i in range(3):
    print(f"RMS of RIR = {rms_levels[i]}")

(1000, 4, 2)
RMS of RIR = [[7.4201950e-04 2.7637041e-05]
 [7.1022433e-04 1.5197927e-05]
 [1.0016420e-03 1.5790252e-05]
 [6.0776662e-04 1.6449338e-05]]
RMS of RIR = [[7.36104092e-04 2.63096808e-05]
 [7.06433610e-04 1.57759641e-05]
 [9.99839045e-04 1.67664130e-05]
 [6.06665155e-04 1.44089845e-05]]
RMS of RIR = [[7.3734019e-04 3.5381654e-05]
 [7.0538954e-04 1.5878721e-05]
 [9.9737174e-04 1.5951553e-05]
 [6.0618843e-04 3.1575102e-05]]


### Using 'raw' datas
If you use this cell, the whole data from dataset will be used for Linear Regression

In [5]:
# Data from first cell np.load('...../Human1/deconvoled_trim.npy')  
features = human_rir_data  

### Using datas then extract feature using RMS
If you use this cell, the whole data from dataset will be used.
Then the Root Mean Squared (RBS) formula will be applied to it.
This will reduce the whole sampling to an unique scalar.

By default, the number of values kept is set to 2. 
Please change this number in the cell just above, where "rms_over_n_partitions" function is defined.

In [6]:
# Normalize the RMS levels
features = (rms_levels - np.mean(rms_levels, axis=0)) / np.std(rms_levels, axis=0)

### Using datas with PCA
If you use this cell, the whole data from dataset will be used.
Then, a PCA Linear dimensionality reduction using Singular Value Decomposition of the data to a lower dimensional space will be applied.


In [9]:
features = human_rir_data.reshape((1000,-1)) # 1000 is the N_datapoints; 104 for human2 dataset
pca = PCA(n_components=10)
features = pca.fit_transform(features)

We now have a set of features, and a set of labels (loaded in the first cells).
We can then use the Linear Regression model to make predictions.

In [10]:
labels = human_rir_labels

# Classical approach of a Linear Regression model.

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape the RMS levels data to have the correct shape
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Train a linear regression model for each coordinate separately 
regressor_x = LinearRegression()
print("model is fitting for x coordinates")
regressor_x.fit(X_train_reshaped, y_train[:, 0])  # Train for x-coordinate

regressor_y = LinearRegression()
print("model is fitting for y coordinates")
regressor_y.fit(X_train_reshaped, y_train[:, 1])  # Train for y-coordinate

# Reshape X_test to have only two dimensions
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Make predictions on test set
y_pred_x = regressor_x.predict(X_test_reshaped)
y_pred_y = regressor_y.predict(X_test_reshaped)


# Select 10 random indices from the test set
random_indices = random.sample(range(len(y_test)), 3)

# Combine x and y predictions into coordinates
y_pred = np.column_stack((y_pred_x, y_pred_y))

# Print the predicted and true positions for the selected indices
print("Randomly Selected Predictions:")
for idx in random_indices:
    true_pos = y_test[idx]
    pred_pos = y_pred[idx]
    print(f"Index {idx}: True Position = {true_pos}, Predicted Position = {pred_pos}")

# Calculate Euclidean distance between predicted and actual coordinates
errors = np.sqrt(np.sum((y_pred - y_test)**2, axis=1))
mean_error = np.mean(errors)
print(f'\nMean localization error: {mean_error:.3f} mm')

model is fitting for x coordinates
model is fitting for y coordinates
Randomly Selected Predictions:
Index 169: True Position = [-3742.64284046 -2567.70120661], Predicted Position = [-2881.94  -2179.181]
Index 141: True Position = [-2635.02066166 -1298.19706888], Predicted Position = [-2595.5605 -2224.788 ]
Index 185: True Position = [-150.49529301  762.35079477], Predicted Position = [-1622.9569  -440.9784]

Mean localization error: 1365.305 mm
